# Final Project Submission: Forest Cover Type Classifier

Please fill out:
* Student name: **Kevin McPherson**
* Student pace: self paced / **part time** / full time
* Scheduled project review date/time: **Monday, March 30, 2020, 9:30AM**
* Instructor name: **Abhineet Kulkarni**
* Blog post URL:


## The Data

The data for this project was supplied from the University of California, Irvine's Machine Learning Repository.

The data is a collection of cartographic features of 30 x 30 square meter forest patches in the Roosevelt National
Forest in Northern Colorado. 

The objective for this data is simple: Are we able to classify forest cover (i.e., tree) type based on this cartographic data

## Step 1: Load and Describe the Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

The dataset originally comes from 

In [2]:
cover = pd.read_csv('covtype.csv') # read in data

FileNotFoundError: [Errno 2] File b'covtype.csv' does not exist: b'covtype.csv'

In [ ]:
print('Data Dimensions:')
print('Number of Records:', cover.shape[0])
print('Number of Features:', cover.shape[1])  # print shape

In [ ]:
print('Feature Names:') # print feature names
print(cover.columns)

In [ ]:
print(cover.info())

In [ ]:
pd.set_option('display.max_columns', 999)

Looking over the data, we can tell that there is a good mix of geographical features about the forest samples, even if the target variable types are not well represented.

For instance, look at the range in horizontal distance to roadways.

In [ ]:
cover.describe()

In [ ]:
cover.isnull().sum()

In [ ]:
print('Skewness of the below features:')
print(cover.skew())

In [ ]:
skew = cover.skew()
skew_df = pd.DataFrame(skew,index=None,columns=['Skewness of Feature'])
plt.figure(figsize=(15,7))
sns.barplot(x=skew_df.index, y = 'Skewness of Feature',data= skew_df)
plt.xticks(rotation=90)

In [ ]:
class_dist = cover.groupby('Cover_Type').size()
class_label = pd.DataFrame(class_dist,columns=['Size'])
plt.figure(figsize=(8,6))
sns.barplot(x = class_label.index,y = 'Size', data = class_label)
plt.title('Cover Type Raw Sum in Roosevelt National Forest')

In [ ]:
for i,number in enumerate(class_dist):
    percent=(number/class_dist.sum())*100
    print('Cover_Type',class_dist.index[i], ':', '%.2f'% percent, '% of the data.')

In [ ]:
cont_data=cover.loc[:,'Elevation':'Horizontal_Distance_To_Fire_Points']
binary_data=cover.loc[:,'Wilderness_Area1':'Soil_Type40']
Wilderness_data=cover.loc[:,'Wilderness_Area1': 'Wilderness_Area4']
Soil_data=cover.loc[:,'Soil_Type1':'Soil_Type40']

In [ ]:
for col in binary_data:
    count=binary_data[col].value_counts()
    print(col,count)

## Steps 2 and 3: Exploratory Data Analysis and Feature Engineering

### Exploring the Data

In [ ]:
cover['Cover_Type']=cover['Cover_Type'].astype('category')

for i, col in enumerate(cont_data.columns):
    plt.figure(i,figsize=(8,4))
    sns.violinplot(x=cover['Cover_Type'], y=col, data=cover, palette="coolwarm")

In [ ]:
for i, col in enumerate(cont_data.columns):
    plt.figure(i)
    sns.distplot(cont_data[col])

In [ ]:
for i, col in enumerate(binary_data.columns):
    plt.figure(i,figsize=(6,4))
    sns.countplot(x=col, hue=cover['Cover_Type'] ,data=cover, palette="rainbow")

### Brief Feature Engineering

One of the interesting features in the data set is 

In [ ]:
cover["Distance_To_Hydrology"] = ( (cover["Horizontal_Distance_To_Hydrology"] ** 2) + 
                              (cover["Vertical_Distance_To_Hydrology"] ** 2) ) ** (0.5) # do Pythagorean on Hydrology

In [ ]:
cover.drop(["Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology"], axis=1, inplace=True) # Drop cols

In [ ]:
plt.figure(figsize=(15,8))
sns.heatmap(cont_data.corr(),cmap='magma',linecolor='white',linewidths=1,annot=True)

In [ ]:
cover.drop(["Hillshade_3pm","Hillshade_9am", "Aspect"], axis=1, inplace=True) # Drop colinear columns

## Step 4: Fitting to Model and Model Validation

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import (accuracy_score, f1_score, confusion_matrix, 
                             classification_report, plot_confusion_matrix)

### Step 4.1: Train/Test Split

In [ ]:
np.set_printoptions(precision=2) # Set precision for the upcoming confusion matrices

In [ ]:
# Get a feature names array going for future graphical listings
feature_names = cover.columns.tolist()
feature_names.remove('Cover_Type')
feature_names = np.array(feature_names)

In [ ]:
y = cover.Cover_Type # target variable

In [ ]:
X = cover.drop('Cover_Type', axis=1) # features variable

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, random_state=123) # Standard test/train split

### Step 4.2: Logisitic Regression Classifier

In [ ]:
log_clf = LogisticRegression(C=0.01, random_state=123, solver='liblinear', max_iter = 200)
log_model = log_clf.fit(X_train, y_train)

In [ ]:
coefs = np.abs(log_clf.coef_[0])
indices = np.argsort(coefs)[::-1]

In [ ]:
plt.figure()
plt.title("Feature importances (Logistic Regression)")
plt.bar(range(10), coefs[indices[:10]],
       color="r", align="center")
plt.xticks(range(10), feature_names[indices[:10]], rotation=45, ha='right')
plt.subplots_adjust(bottom=0.3)

### Step 4.3: Random Forest Classifier

In [ ]:
forest_clf = RandomForestClassifier()
forest_model = forest_clf.fit(X_train, y_train)

In [ ]:
coefs = forest_clf.feature_importances_
indices = np.argsort(coefs)[::-1]

In [ ]:
plt.figure()
plt.title("Feature importances (Random Forests)")
plt.bar(range(10), coefs[indices[:10]],
       color="r", align="center")
plt.xticks(range(10), feature_names[indices[:10]], rotation=45, ha='right')
plt.subplots_adjust(bottom=0.3)

### Step 4.4: K-Nearest Neighbors Classifier

In [ ]:
neighbors = np.arange(1,10)
train_accuracy =np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

for i,k in enumerate(neighbors):
    #Setup a knn classifier with k neighbors
    knn = KNeighborsClassifier(n_neighbors=k)
    
    #Fit the model
    knn.fit(X_train, y_train)
    
    #Compute accuracy on the training set
    train_accuracy[i] = knn.score(X_train, y_train)
    
    #Compute accuracy on the test set
    test_accuracy[i] = knn.score(X_test, y_test) 

In [ ]:
plt.figure(figsize=(10,6))
plt.title('k-NN Varying number of neighbors')
plt.plot(neighbors, test_accuracy, label='Testing Accuracy')
plt.plot(neighbors, train_accuracy, label='Training accuracy')
plt.legend()
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
knn_model = knn_clf.fit(X_train,y_train)

## Step 5: Model Validation

### Step 5.1: Logistic Regression Validation

In [ ]:
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(log_clf, X_test, y_test,
                                 display_labels=y.unique(),
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

In [ ]:
log_training_preds = log_clf.predict(X_train)
log_training_accuracy = accuracy_score(y_train, log_training_preds)

log_val_preds = log_clf.predict(X_test) # y_hat
log_val_accuracy = accuracy_score(y_test, log_val_preds)

print('Logistic Regression Training accuracy: {:.4}%'.format(log_training_accuracy*100) )
print('Logistic Regression Testing accuracy: {:.4}%'.format(log_val_accuracy*100) )

In [ ]:
print(classification_report(y_test, log_val_preds))

### Step 5.2: Random Forest Validation

In [ ]:
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(forest_clf, X_test, y_test,
                                 display_labels=y.unique(),
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

In [ ]:
forest_training_preds = forest_clf.predict(X_train)
forest_training_accuracy = accuracy_score(y_train, forest_training_preds)

forest_val_preds = forest_clf.predict(X_test) # y_hat
forest_val_accuracy = accuracy_score(y_test, forest_val_preds)

print("Forest Training Accuracy: {:.4}%".format(forest_training_accuracy * 100))
print("Forest Testing accuracy: {:.4}%".format(forest_val_accuracy * 100))

In [ ]:
print(classification_report(y_test, forest_val_preds))

### Step 5.3: KNN Validation

In [ ]:
titles_options = [("Confusion matrix, without normalization", None),
                  ("Normalized confusion matrix", 'true')]
for title, normalize in titles_options:
    disp = plot_confusion_matrix(knn_clf, X_test, y_test,
                                 display_labels=y.unique(),
                                 cmap=plt.cm.Blues,
                                 normalize=normalize)
    disp.ax_.set_title(title)

    print(title)
    print(disp.confusion_matrix)

In [ ]:
knn_training_preds = knn_clf.predict(X_train)
knn_training_accuracy = accuracy_score(y_train, knn_training_preds)

knn_val_preds = knn_clf.predict(X_test) # y_hat
knn_val_accuracy = accuracy_score(y_test, knn_val_preds)

print("KNN Training Accuracy: {:.4}%".format(knn_training_accuracy * 100))
print("KNN Testing accuracy: {:.4}%".format(knn_val_accuracy * 100))

In [ ]:
print(classification_report(y_test, knn_val_preds))

## Summary

**The best model for cover classification:** Random Forest

The model was able to best predict forest cover type with 95% accuracy, and had precision and recall all above 80%.

K-Nearest Neighbors seemed to perform better in recall but not in precision. 

## Conclusions

I found that the three most important factors to determining cover type, when we classify with a Random Forest
Classifier are: 1) Elevation, 2) Distance to Roadways, and 3) Distance to fire points. These makes sense as the distance to fire points 
will ultimately determine the soil type that develops there and the other flora and fauna that can flourish in and around the environ-
ment to support the cover type. In addition, certain tree cover will be better suited to survive near roadways and at certain elevations.

## Future Directions

- **Class imbalance**: One of the biggest features of this dataset is class imbalance. Specifically, the classes of the target variable `Cover_Type` that are over-represented in the data are cover types 1 and 2, or Spruce/Fir and Lodgepole Pine are the most common target classes in this dataset. Working on some way of reducing the disparity, either by doing a _cost sensitive classifier_ if there's no way of resampling the data, or using some sort of synthetic sampling technique like SMOTE might help.
- **Applying SVM**: Although it appears that Random Forest is a great way to predict what cover type exists, it would be interesting to see how a "one vs one" or even a "one vs all" multi-clas classifier would work in this situation. Thus, applying SVM would be an interesting way of bringing in new classification technology to the project.
- **Feature Engineering**: most of the data contains "vertical" or "horizontal distances to certain points, but we know from experience that the Earth lies on certain curvatures that don't make just pure vertical and horizontal distances accurate. Therefore, going back and making Euclidian distances of each feature would be useful. 
- **Protecting against possible overfitting**: I have a high suspicion that my current model is overfit. Performing cross-validation is highly desirable.